In [8]:
from openai import OpenAI
import os

In [9]:
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

In [10]:
openai = OpenAI(
    api_key = os.environ.get("OPENAI_API_KEY"),
    base_url='http://127.0.0.1:1234/v1'
)

In [11]:
# функция получения продолжение фразы (для не инструктивной модели)
def generate_completion(prompt, model, max_tokens):
    completion=openai.completions.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        stop="<--->"
    )
    return completion.choices[0].text

In [12]:
# получение датасетов из файла
import pandas as pd
import pyarrow
dataset=pd.read_parquet('test-wmt16.parquet', engine='pyarrow')
dataset_train=pd.read_parquet('train-wmt16.parquet', engine='pyarrow')

In [13]:
# функция получения ответа инструктивной модели
def generate_instruct_response(model, messages):
    completion=openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0
    )
    return completion.choices[0].message.content

In [14]:
# библиотека для bleu
import nltk

In [9]:
results_instruct_en_ru=[]
for i in range(10):
    result=generate_instruct_response(
        model='qwen2.5-3b-instruct',
        messages=[
            {'role': 'system', 'content': 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.'},
            {'role': 'user', 'content': f'Translate to Russian: {dataset_train['translation'][0]['en']}'},
            {'role': 'assistant', 'content': dataset_train['translation'][0]['ru']},
            {'role': 'user', 'content': f'Translate to Russian: {dataset_train['translation'][1]['en']}'},
            {'role': 'assistant', 'content': dataset_train['translation'][1]['ru']},
            {'role': 'user', 'content': f'Translate to Russian: {dataset_train['translation'][2]['en']}'},
            {'role': 'assistant', 'content': dataset_train['translation'][2]['ru']},
            {'role': 'user', 'content': f'Translate to Russian: {dataset['translation'][i]['en']}'}
        ],
    )
    reference = dataset['translation'][i]['ru'] 
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference.split()], result.split())
    results_instruct_en_ru.append([result, reference, BLEUscore])
    print(results_instruct_en_ru[i])

C:\Python312\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Python312\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


['Сocial卡片居民伊万诺沃地区的应被认作电子支付工具。看起来这里有一个翻译上的混淆，因为“social card”通常指的是社会福利卡或身份识别卡，并不是一种可以用于电子支付的设备。正确的翻译应该是：\n\n"Карта социального обеспечения residents of Ivanovo region должна быть признана электронным средством оплаты."', 'Социальная карта жителя Ивановской области признается электронным средством платежа.', 4.65988169467104e-155]


C:\Python312\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


['Эти изменения учитывали сохранение скидочных проездов на общественном транспорте.', 'Изменения предусматривают сохранение льготы на проезд в общественном пассажирском транспорте.', 1.331091014027761e-231]
['Начиная с 24 сентября, организация скидочных путевок для граждан региона Иваново при использовании социальной карты изменится.', 'С 24 сентября меняется порядок организации льготного проезда граждан с использованием социальной карты жителя Ивановской области.', 5.667113443314699e-155]
['Служба пресса для правительства России по Ивановской области сообщает, что изменения учитывали сохранение скидочных тарифов на общественный транспорт и вводят систему взаимных расчетов через социальную карту согласно требованиям Федерального закона № 161 «О Национальной платежной системе».', 'Как сообщает пресс-служба правительство России по Ивановской области, изменения предусматривают сохранение льготы на проезд в общественном пассажирском транспорте и приводят схему взаиморасчетов по социальной 

In [19]:
for i in range(2500, len(dataset['translation'])):
    result=generate_instruct_response(
        model='qwen2.5-3b-instruct',
        messages=[
            {'role': 'system', 'content': 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.'},
            {'role': 'user', 'content': f'Translate to Russian: {dataset_train['translation'][0]['en']}'},
            {'role': 'assistant', 'content': dataset_train['translation'][0]['ru']},
            {'role': 'user', 'content': f'Translate to Russian: {dataset_train['translation'][1]['en']}'},
            {'role': 'assistant', 'content': dataset_train['translation'][1]['ru']},
            {'role': 'user', 'content': f'Translate to Russian: {dataset_train['translation'][2]['en']}'},
            {'role': 'assistant', 'content': dataset_train['translation'][2]['ru']},
            {'role': 'user', 'content': f'Translate to Russian: {dataset['translation'][i]['en']}'}
        ],
    )
    reference = dataset['translation'][i]['ru'] 
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference.split()], result.split())
    results_instruct_en_ru.append([result, reference, BLEUscore])
    print(i, results_instruct_en_ru[i])

2500 ['Здесь приведен отчёт о матче и реакция на него.', 'Вот отчет о матче и реакция на игру.', 0.44632361378533286]
2501 ['Ангел Ди Мария и Эден Азербайчан были наtargetах, как Парижский Сент-Этьен начали свой турнир в Лиге чемпионов с уверенной 2:0 победой над Мальмё в Парк-де-Принце во вторник. \n\nNote: There seems to be a typo or error in the original text, particularly with "target" and "Edinson Cavani". The correct names are Angel Di Maria and Edinson Cavani (or Edinson Cavani). Also, "Malmo" is likely meant to be "Malmö", which is the Swedish name for Malmo. \n\nThe corrected sentence would be:\n"Ангел Ди Мария и Эден Азербайчан были на targetах, как Парижский Сент-Этьен начали свой турнир в Лиге чемпионов с уверенной 2:0 победой над Мальмё в Парк-де-Принце во вторник."', 'Анхель ди Мария и Эдинсон Кавани сделали свое дело, когда "Пари Сен-Жермен" начала свою игру в Лиге Чемпионов с крайне удобной победы 2:0 против "Мальмё" на стадионе Парк де Пренс во вторник.', 3.29488867815

In [20]:
f = open('result_instruct_en_ru.txt', 'w', encoding="utf-8")
f.write(str(results_instruct_en_ru))

910938

In [1]:
f = open('result_instruct_en_ru.txt', 'r', encoding="utf-8")
results_instruct_en_ru=f.read()
print(results_instruct_en_ru)

[['Сocial卡片居民伊万诺沃地区的应被认作电子支付工具。看起来这里有一个翻译上的混淆，因为“social card”通常指的是社会福利卡或身份识别卡，并不是一种可以用于电子支付的设备。正确的翻译应该是：\n\n"Карта социального обеспечения residents of Ivanovo region должна быть признана электронным средством оплаты."', 'Социальная карта жителя Ивановской области признается электронным средством платежа.', 4.65988169467104e-155], ['Эти изменения учитывали сохранение скидочных проездов на общественном транспорте.', 'Изменения предусматривают сохранение льготы на проезд в общественном пассажирском транспорте.', 1.331091014027761e-231], ['Начиная с 24 сентября, организация скидочных путевок для граждан региона Иваново при использовании социальной карты изменится.', 'С 24 сентября меняется порядок организации льготного проезда граждан с использованием социальной карты жителя Ивановской области.', 5.667113443314699e-155], ['Служба пресса для правительства России по Ивановской области сообщает, что изменения учитывали сохранение скидочных тарифов на общественный транспорт и вводят систему в

In [2]:
results_instruct_en_ru=eval(results_instruct_en_ru)

In [10]:
results_instruct_7B_en_ru=[]
with open('result_instruct_7B_en_ru.txt', 'r', encoding="utf-8") as f:
    for line in f:
        index=line.find(" ")
        element=line[(index+1):]
        results_instruct_7B_en_ru.append(eval(element))
print(results_instruct_7B_en_ru[0])

7.746339753396389e-155


In [1]:
results_instruct_14B_en_ru=[]
with open('result_instruct_14B_en_ru.txt', 'r', encoding="utf-8") as f:
    for line in f:
        index=line.find(" ")
        element=line[(index+1):]
        results_instruct_14B_en_ru.append(eval(element))
print(results_instruct_14B_en_ru[0])

['Социальная карта жителей Ивановской области должна быть признана как электронный платежный инструмент.', 'Социальная карта жителя Ивановской области признается электронным средством платежа.', 7.401184483348608e-155]


In [5]:
results_instruct_ru_en=[]
with open('result_instruct_ru_en.txt', 'r', encoding="utf-8") as f:
    for line in f:
        index=line.find(" ")
        element=line[(index+1):]
        results_instruct_ru_en.append(eval(element))
print(results_instruct_ru_en[0])

['The social card for residents of Ivanovo Region is recognized as an electronic payment method.', 'The social card of residents of Ivanovo region is to be recognised as an electronic payment instrument.', 0.27251836515605526]


In [8]:
results_instruct_7B_ru_en=[]
with open('result_instruct_7B_ru_en.txt', 'r', encoding="utf-8") as f:
    for line in f:
        index=line.find(" ")
        element=line[(index+1):]
        results_instruct_7B_ru_en.append(eval(element))
print(results_instruct_7B_ru_en[0])

['The social card for residents of Ivanovo Oblast is recognized as an electronic payment method.', 'The social card of residents of Ivanovo region is to be recognised as an electronic payment instrument.', 0.27251836515605526]


In [12]:
results_instruct_14B_ru_en=[]
f = open('result_instruct_14B_ru_en.txt', 'r', encoding="utf-8")
results_instruct_14B_ru_en=f.read()
results_instruct_14B_ru_en=eval(results_instruct_14B_ru_en)
print(results_instruct_14B_ru_en[0])

['The Social Card of an Ivanovo Oblast resident is recognized as an electronic means of payment.', 'The social card of residents of Ivanovo region is to be recognised as an electronic payment instrument.', 0.24677534696560227]


In [4]:
summa=0
for i in range(len(results_instruct_en_ru)):
    summa+=results_instruct_en_ru[i][2]
average=summa/len(results_instruct_en_ru)
print(average)

0.09058653192737136


In [11]:
summa=0
for i in range(len(results_instruct_7B_en_ru)):
    summa+=results_instruct_7B_en_ru[i][2]
average=summa/len(results_instruct_7B_en_ru)
print(average)

0.12369258973648035


In [13]:
summa=0
for i in range(len(results_instruct_14B_en_ru)):
    summa+=results_instruct_14B_en_ru[i][2]
average=summa/len(results_instruct_14B_en_ru)
print(average)

0.31520877522360685


In [14]:
summa=0
for i in range(len(results_instruct_ru_en)):
    summa+=results_instruct_ru_en[i][2]
average=summa/len(results_instruct_ru_en)
print(average)

0.3645407624712958


In [15]:
summa=0
for i in range(len(results_instruct_7B_ru_en)):
    summa+=results_instruct_7B_ru_en[i][2]
average=summa/len(results_instruct_7B_ru_en)
print(average)

0.3712062875712948


In [16]:
summa=0
for i in range(len(results_instruct_14B_ru_en)):
    summa+=results_instruct_14B_ru_en[i][2]
average=summa/len(results_instruct_14B_ru_en)
print(average)

0.3739398647922025


In [51]:
print(generate_completion(model='qwen2.5-3b', prompt=f'en: {dataset[0]['en']}. ru:', max_tokens=30))

BadRequestError: Error code: 400 - {'error': 'Field with key llm.prediction.stopStrings does not satisfy the schema:[\n  {\n    "code": "too_small",\n    "minimum": 1,\n    "type": "string",\n    "inclusive": true,\n    "exact": false,\n    "message": "String must contain at least 1 character(s)",\n    "path": [\n      0\n    ]\n  }\n]'}